<span type="title">从电路到加法运算</span> | <span type="update">2018-07-13</span>

<span type="intro"><p type="card-text">本章主要介绍算术逻辑单元的电路及其实现。在最开始介绍了整体的CPU使用ALU运算的流程，之后介绍了ALU运算所依赖的两个重要电路部件：构成ALU运算的门电路和构成寄存器存储能力的触发器，之后将其整合，介绍了ALU运算流程的内部电路实现。之后，以算术运算举例，介绍了加法和减法在ALU中的两种不同的实现：行波进位和超前进位加法器。</p></span>

# ALU运算流程

以MIPS为例，进行 `add $8,$9,$10` 的加法运算时，首先代码被IR读入到指令译码中，之后控制电路向ALU发出add的操作指令，之后，在内部总线上实现R9和R10和ALU两个输入的连接，以及R8和ALU输出的连接。这样ALU就有了输入和输出，并且进行正常的运算了。

![](w3p1.png)

常见的R型算术运算指令有 `add addu sub subu`。其中不带u的可能造成溢出，而带有u的会自动忽略溢出（unsigned）。

对于 `addi %21,$22,-50` 这条I型指令而言，唯一的不同是，接入ALU的一个是IR提供的立即数，而另一个是R22寄存器。

同样的，I型也含有 `addi addiu` 两种不同实现。其中，因为寄存器中的数和立即数的位数不同，前者为8位，后者为16位，因此需要进行算术扩展：`R[rt]=R[rs]+SignExtImm` 这样的话就可以进行加法运算了。

对于逻辑运算指令，有 `and or nor andi ori`，其中带有i的是I型指令，对于I型指令而言，也存在立即数和寄存器位数不同问题，需要进行0补齐：`R[rt]=R[rs]|(&)ZeroExtImm`

# ALU电路基础：门电路

现代集成电路中经常使用MOS(Metal Oxide Semiconductor 金属氧化物半导体)晶体管。其中，由PMOS和NMOS共同组成互补型MOS集成电路（Complementary MOS）,其中NMOS和PMOS原理和示意图如下，Source表示来源，Drain表示出口，Gate表示控制阀，NMOS遇到高电平1则通，而PMOS遇到低电平0则通。

![](w3p2.png)

## 非门 NOT

![](w3p3.png)

非门的逻辑符号、函数表示、原理如图所示，VDD表示Source，高电平，而接地线表示低电平和Drain，A为1时，代表高电平，则PMOS不通，而NMOS通，所以电流不经过PMOS，因此Y为0.反之，如果A为0，则PMOS通，Source的高电平流入Y，Y为1。


## 与门 AND

如下图所示，与采用如下表示。

![](w3p4.png)

其中，实际用与非门和非门实现与门。其工作过程如下：

![](w3p5.png)


## 或门 OR

或门如下表示：

![](w3p6.png)

## 异或门 OR

异或门如下表示，其代表含义为，如果两个值不相同，则为真，否者为假。

![](w3p7.png)

综上所述，逻辑门和晶体管的关系，以及和MIPS汇编语言的对应如下：

![](w3p8.png)

# 寄存器电路基础：触发器

寄存器内部结构如下：

![](w3p9.png)

其中每一个字节都是由一个DFF构成，D触发器结构简单，是存储信息的基本单元，其蛀牙有一个数据输入、输出和时钟输入。在时钟的上升沿0->1，采样输入D的值，传送到输出Q，在其余时间，输出Q的值保持不变。

D flip-flop 就相当于一个对显示器（Input）拍照的相机，拍完照片后传送给下一台显示器（Output）。时钟端代表着快门，Setup/Hold时间代表被拍摄对象的保持时间。在时钟间隔，被拍摄对象的改变不会引起相机的反应，也不会在output端口表现出来。

![](w3p10.png)

如上图所示，第一条红线部分，这时候传入有消息，但那时clock没有到点，所以没有output的改变，在第二条红线附近，传入改变为0，clock到点，因此经过短暂的拍摄耗时，output的内容改变了。

# ALU电路流程

![](w3p11.png)

如上图所示，每一个寄存器中的一个字节都是通过DFF来实现的（rs和rt，每一个都由32个DFF组成）。而ALU则是通过多个不同的逻辑门来实现的，这些DFF连接到ALU的两个端口。

ALU是多功能算术运算单元，其会同时开始多个运算，而我们只需要根据指令选择其中一个运算结果即可。注意的是，对于每种运算，每一位的运算都对应一个逻辑门（一个DFF对应一个逻辑门，要完成32位运算，则有32个DFF对应32个逻辑门）。

![](w3p12.png)

如上图所示，clock控制r8 r9 r10的电路，传输信号到ALU，ALU从中进行运算，选择op指令，输出到r8中。

# 加法器和减法器

## 电路基础：半加器和全加器

半加器的功能是将两个一位二进制相加，其输入为A和B，输出为S和C，其中S负责计算结果，C负责进位。

![](w3p13.png)

对于全加器而言，其由两个半加器组成，可以处理来自上一次运算的进位信息Cin，进位输出放在Cout中，本位结果放在S中。

![](w3p14.png)

## 加法运算概要

如下所示，由32个全加器构成的ALU，其中A和B的32位中的每一位都对应一个全加器，经过进位和运算输出，结果保存在S和C中。

![](w3p15.png)



## 运算溢出的处理

overflow 是一种运算结果超出了正常表示范围的现象，其只针对有符号数的运算，表现为，两个正数相加结果为负数，两个负数相加，结果为正数。

进位可以看作无符号数的溢出，有溢出不一定有进位，有进位也不一定有溢出。

**在现实情况下，采用将最高位的Cin和Cout接到一个异或逻辑门中，就可以检测是否发生了溢出。**

对于MIPS，溢出的处理方式是：如果将操作数看作有符号数，那么发生溢出时产生异常(add,addi)，如果将操作看作无符号数，则不处理溢出（addu,addiu）。

对于x86而言，溢出有Overflow Flag，在标志寄存器中的第11位会进行标记，如果有则标记为1，否则为0.

## 减法运算概要

减法就是负数的加法，区别是，二进制负数次啊用 `按位取反，末位加1` 的规则进行，如下图所示，在输入前接入一个判断和非逻辑门即可。

![](w3p15.png)

## 加法器的种类和优化

上述介绍的加法器可以称作 Ripple Carry Adder，RCA，行波进位加法器，其结构简单，电路清晰，特点是前一位的Cout连接到后一位的Cin上，缺点是高位的运算必须等待低位的运算结束后才能进行，延时太长。

对于4 BIT RCA 的分析得，总的延时时间等于2（n+1）T = 9T。其中T代表对于一个逻辑门的延时，此外尚未计算线的延时。

![](w3p17.png)

对于32 BIT RCA 而言，这种延时达到了 65T，对于28nm工艺，T约为0.02ns，那么32 Bit RCA的时钟频率最高可以为769MHZ，这显然是无法接受的（iPhone 5s A7 Soc 28nm制造工艺，主频1.3Ghz）

1Ghz意味着每秒可以产生1亿次电脉冲，也意味着1/1ns周期迭代一次。

Ci+1 = (Ai·Bi)+(Ai·Ci)+(Bi·Ci) = (Ai·Bi)+(Ai+Bi)·Ci

设生成信号 Gi = Ai·Bi，传播信号 Pi = Ai+Bi，则 Ci+1 = Gi + Pi·Ci

![](w3p18.png)

根据此，可以提前计算高位的电路。

![](w3p19.png)

但是实际上，实现32位的计算电路是极其复杂的，因此 Carry Lookahead Adder，CLA，超前进位加法器采用了将行波进位加法器和这种加法器结合起来的方法，将多个RCA通过CLA进行输入，然后分别计算输出，因为每级的进位输入都是立即通过CLA得到的，而计算C3在CLA中需要3级延迟，而最后一级全加器还需要1级延迟（C4，S？），因此有4级延迟。